In [1]:
import sys
sys.path.append('.')
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import torch
%matplotlib inline

## Train Data

In [2]:
train_df = pd.read_csv('TRAIN.txt', sep='\t')
print('{0:,} registros a las {1}'.format(len(train_df), datetime.now()))
train_df.head()

60,797 registros a las 2020-10-05 23:39:33.024163


,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
0,50833,2253302,0,S,2,1,192034,58.16,2006,40,34.52,46.58
1,67770,700754,0,S,2,1,3891897,30.95,2004,20,30.99,16.98
2,71253,528795,0,S,2,1,245070,49.99,2010,32,14.01,30.98
3,31295,901250,0,D,4,1,3480889,390.00,2004,21,0.00,73.13
4,43339,578450,0,S,5,1,1422247,51.95,1990,35,4.99,35.99


In [3]:
train_df.sort_values(['ORDEN', 'SKU'], ascending=[True, True], inplace=True)
print('{0:,} registros a las {1}'.format(len(train_df), datetime.now()))
train_df.head()

60,797 registros a las 2020-10-05 23:40:46.538532


,ORDEN,SKU,VENTA_ALTA,MARKETING,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
58481,2,1737127,0,D,15,1,682743,44.99,2015,8,28.97,31.84
42622,3,3255963,0,D,7,1,1016014,24.81,2005,39,0.00,15.54
19535,4,612701,0,D,0,0,340464,46.00,2013,34,30.19,27.97
13680,6,115883,1,D,4,1,334011,100.00,2006,20,133.93,83.15
11324,7,863939,1,D,2,1,1287938,121.95,2010,28,4.00,23.99


Articulos por orden??

In [4]:
grouped_1_df = train_df.groupby('ORDEN').agg({'SKU':'nunique'}).reset_index()
grouped_1_df

,ORDEN,SKU
0,2,1
1,3,1
2,4,1
3,6,1
4,7,1
...,...,...
60792,85101,1
60793,85102,1
60794,85103,1
60795,85104,1


In [5]:
grouped_1_df['SKU'].value_counts()

1    60797
Name: SKU, dtype: int64

In [6]:
train_df.isna().sum()

ORDEN                0
SKU                  0
VENTA_ALTA           0
MARKETING            0
ACTUALIZACIONES      0
ACTUALIZADO          0
VENTAS_ACUMULADAS    0
PRICE                0
LANZAMIENTO          0
CANTIDAD             0
PRECIO_MIN           0
SD_PRECIO            0
dtype: int64

In [7]:
train_df.describe()

,ORDEN,SKU,VENTA_ALTA,ACTUALIZACIONES,ACTUALIZADO,VENTAS_ACUMULADAS,PRICE,LANZAMIENTO,CANTIDAD,PRECIO_MIN,SD_PRECIO
count,60797.000000,6.079700e+04,60797.000000,60797.000000,60797.000000,6.079700e+04,60797.000000,60797.000000,60797.000000,60797.000000,60797.000000
mean,41643.421633,6.529548e+05,0.170699,4.144497,0.788411,1.219195e+06,98.963463,2005.984785,43.821965,56.680905,47.288439
std,25057.005884,6.875584e+05,0.376249,3.945417,0.408439,1.539134e+06,79.316429,6.107125,36.645181,106.498309,132.424575
min,2.000000,5.000100e+04,0.000000,0.000000,0.000000,1.250000e+02,0.000000,1935.000000,0.000000,0.000000,0.000000
25%,19462.000000,1.701200e+05,0.000000,2.000000,1.000000,2.431360e+05,49.950000,2003.000000,22.000000,20.400000,18.760000
50%,41498.000000,5.409390e+05,0.000000,3.000000,1.000000,7.117010e+05,78.950000,2007.000000,34.000000,43.990000,36.060000
75%,63563.000000,7.589470e+05,0.000000,6.000000,1.000000,1.550652e+06,127.950000,2010.000000,53.000000,79.290000,56.940000
max,85105.000000,3.959831e+06,1.000000,99.000000,1.000000,1.666966e+07,3986.310000,2017.000000,1523.000000,14140.210000,19138.790000


In [4]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)
B.shape

(4, 1)